In [ ]:
!pip install -qq transformers

In [ ]:
# import pandas as pd
# df = pd.read_csv('/content/drive/MyDrive/political_leaning/final_final_version.csv')

In [ ]:
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict,abc
from textwrap import wrap
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from pymongo import MongoClient
%matplotlib inline
%config InlineBackend.figure_format='retina'
sns.set(style='whitegrid', palette='muted', font_scale=1.2)
HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]
sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))
rcParams['figure.figsize'] = 12, 8
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
# !pip install emoji

In [ ]:
!pip install dnspython

In [ ]:
import tweepy as tw
consumer_key = 'XXXXXXXXXXXXXXXXXXXXXXXX'
consumer_secret = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
access_token = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
access_token_secret='XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
# import emoji
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)
client = MongoClient("connection_string",27017)
political_leaning = client.political_leaning
leader_tweets = political_leaning.leader_tweets

In [ ]:
!curl ipecho.net/plain

34.122.180.100

In [ ]:
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [ ]:
conn = psycopg2.connect(
   database="postgres", user='ubuntu', password='xxxxxxxxxxxxxxxxxx', host='xx.xxx.xx.xxx', port= '5432'
)
cursor = conn.cursor()
checker_table = 'tblPoliticalLeaning'
table_name = 'tblUserInfo'

In [ ]:
class_names = ['Democratic','Republican']

In [ ]:
PRE_TRAINED_MODEL_NAME = 'bert-base-cased'
MAX_LEN=100

In [ ]:
bert_model = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
class PoliticalLeaningClassifier(nn.Module):
  def __init__(self,n_classes):
    super(PoliticalLeaningClassifier, self).__init__()
    self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
    self.drop = nn.Dropout(p=0.3)
    self.out = nn.Linear(self.bert.config.hidden_size,n_classes)
  def forward(self,input_ids,attention_mask):
    _,pooled_output = self.bert(input_ids=input_ids,attention_mask = attention_mask,return_dict=False)
    output = self.drop(pooled_output)
    return self.out(output)

In [ ]:
model = PoliticalLeaningClassifier(len(class_names))

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
model.load_state_dict(torch.load('/content/drive/MyDrive/political_leaning/largers_batch_size/BERT_classifier_0.8061427837699024_Political_leaning.pt'))

<All keys matched successfully>

In [ ]:
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [ ]:
model = model.to(device)

In [ ]:
for i in range(leader_tweets.estimated_document_count()):
  temp = leader_tweets.find_one()
  prediction_list=[]
  if temp['Tweets']!=[]:
    for t in temp['Tweets']:
      encoded_review = tokenizer.encode_plus(
          t,
          max_length=MAX_LEN,
          add_special_tokens=True,
          return_token_type_ids=False,
          pad_to_max_length=True,
          return_attention_mask=True,
          return_tensors='pt',
      )
      input_ids = encoded_review['input_ids'].to(device)
      attention_mask = encoded_review['attention_mask'].to(device)
      output = model(input_ids, attention_mask)
      _, prediction = torch.max(output, dim=1)
      prediction_list.append(class_names[prediction])
    democratic_tweets = (np.sum(np.array(prediction_list)=='Democratic')/len(prediction_list))*100
    republican_tweets = (np.sum(np.array(prediction_list)=='Republican')/len(prediction_list))*100
    if democratic_tweets>republican_tweets:
      leaning = 'Democratic'
    else:
      leaning = 'Republican'
    insert_query = "INSERT INTO "+checker_table+" values("+str(temp['UserId'])+",'"+leaning+"',"+str(democratic_tweets)+","+str(republican_tweets)+")"
    cursor.execute(insert_query)
    conn.commit()
  leader_tweets.find_one_and_delete({'UserId':temp['UserId']})

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2190: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
